In [ ]:
using PAN
using Format: format, printfmtln
using Random: rand
using Optim
using Plots

In [ ]:
netlist = """
ground electrical gnd

E1  x   gnd  vsource     dc=10
R1  x   y    resistor    r={}
I1  y   gnd  JLRESISTOR  resistance={}

model JLRESISTOR nport macro=yes setup="jlres_setup" evaluate="jlres_eval"
"""
R0 = 5.0
netlist = format(netlist, R0, 2*R0);

In [ ]:
netlist_file = "nport.pan"
open(netlist_file, "w") do io
    write(io, netlist)
end;

In [ ]:
jlres_setup() = (resistance = 5.0, );

In [ ]:
function jlres_eval(n::Number, V::AbstractArray, I::AbstractArray, time::Number, parameters...)
    r = parameters[1]
    f = V - r * I
    C = Matrix{Float64}(undef, 1, 1)
    R = Matrix{Float64}(undef, 1, 1)
    C[1,1] = 1.0
    R[1,1] = -r
    return f, C, R
end;

In [ ]:
ok, libs = load_netlist(netlist_file)
if ! ok
    println("load_netlist failed.")
end

In [ ]:
function cost(R)
    cost_id = string(Int32(round(rand() * 1e6)))
    alter(string("Al_", cost_id), "resistance", R[1], libs, instance="I1", invalidate="no", annotate=4)
    V = DC(string("Dc_", cost_id), ["y"], libs, print="yes")
    -V[1]^2 / R[1]
end;

In [ ]:
res = optimize(cost, 0, 100 * R0, GoldenSection());
Ropt = Optim.minimizer(res);

In [ ]:
printfmtln("   Optimal value of resistance: {:.2f} Ohm.", Ropt)
printfmtln("Number of function evaluations: {}.", Optim.iterations(res))

In [ ]:
R = range(R0 / 10, 10 * R0; length=100)
V = DC("Dcsweep", ["y"], libs, start=R[1], stop=R[end], step=diff(R)[1], instance="I1", param="resistance");

In [ ]:
plot(R, V.^2 ./ R, color="black", lw=2, label="")
plot!(Ropt.+zeros(2), [ylims()...], color="red", lw=1, label="Optimal value")
xlabel!("Resistance [Ω]")
ylabel!("Power [W]")